<font size="5">On-Chip Edge Learning</font>

This notebook was executed on the development kit from BrainChip. Furthermore, the on-chip edge learning was executed. The code was inspired by the official code from BrainChip (https://doc.brainchipinc.com/examples/edge/plot_1_edge_learning_kws.html).

In [1]:
import os
from os import listdir
from os.path import isdir, join
import pathlib

import akida
from akida import FullyConnected
from akida import evaluate_sparsity
import cnn2snn
from cnn2snn import check_model_compatibility
from cnn2snn import quantize
from cnn2snn import quantize_layer
from cnn2snn import convert

from math import ceil

from timeit import default_timer as timer
import numpy as np

import itertools
import matplotlib.pyplot as plt

<font size="5"> 1. Load the Data Set and Model</font>

In [2]:
# Load the previous trained model. (Source Model: Final_Edge)

quantized_model = cnn2snn.load_quantized_model('final_quantized_edge_model.h5')

In [3]:
# Define directories of the data

feature_sets_filename = 'final_stored_files_targets_int_normalized_wedge.npz'
feature_sets_filename_edge = 'final_stored_files_targets_int_normalized_edge.npz'

# Load feature sets

feature_sets = np.load(feature_sets_filename)
feature_sets_edge = np.load(feature_sets_filename_edge)
print('Feature Sets: ', feature_sets.files)
print('Feature Sets for Edge: ', feature_sets_edge.files)


Feature Sets:  ['x_train', 'y_train', 'x_val', 'y_val', 'x_test', 'y_test']
Feature Sets for Edge:  ['x_train', 'y_train', 'x_val', 'y_val']


In [4]:
# Assign feature sets of inital data set

x_train = feature_sets['x_train']
y_train = feature_sets['y_train']
x_val = feature_sets['x_val']
y_val = feature_sets['y_val']
x_test = feature_sets['x_test']
y_test = feature_sets['y_test']

# Assign feature sets of edge data set

x_train_edge = feature_sets_edge['x_train']
y_train_edge = feature_sets_edge['y_train']
x_val_edge = feature_sets_edge['x_val']
y_val_edge = feature_sets_edge['y_val']

In [5]:
# CNN for conversion expects (batch, height, width, channels)
# The channels can either be 1 for gray-scaled images or 3 for RGB-images

x_train = x_train.reshape(x_train.shape[0], 
                          x_train.shape[1], 
                          x_train.shape[2], 
                          1)
x_val = x_val.reshape(x_val.shape[0], 
                      x_val.shape[1], 
                      x_val.shape[2], 
                      1)
x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1], 
                        x_test.shape[2], 
                        1)

x_train_edge = x_train_edge.reshape(x_train_edge.shape[0], 
                          x_train_edge.shape[1], 
                          x_train_edge.shape[2], 
                          1)
x_val_edge = x_val_edge.reshape(x_val_edge.shape[0], 
                      x_val_edge.shape[1], 
                      x_val_edge.shape[2], 
                      1)



print('Add dimension to all data: Order Train, Val, Test, Train_edge, Val_edge: ')
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)
print(x_train_edge.shape)
print(x_val_edge.shape)

Add dimension to all data: Order Train, Val, Test, Train_edge, Val_edge: 
(34972, 40, 101, 1)
(4371, 40, 101, 1)
(4371, 40, 101, 1)
(714, 40, 101, 1)
(85, 40, 101, 1)


<font size="5"> 2. Hyperparameter Search</font>

In [6]:
# Define the Akida device

devices = akida.devices()
print('Akida NP available :', devices[0])
device = devices[0]

Akida NP available : <akida.core.HardwareDevice object at 0xffff8807b970>


In [8]:
# Hyperparameter search of "neurons per class"
# In every iteration the summary and floor power of the power measurement is printed

neurons_per_class = [2, 4, 8, 16, 32, 40, 50, 60, 70, 80, 90, 100, 150, 200, 300, 350, 400, 450, 500]
num_labels = 12
num_weights = 300
device.soc.power_measurement_enabled = True

for idx, n in enumerate(neurons_per_class):
    
    akida_model_test = convert(quantized_model)

    akida_model_test.pop_layer()
    layer_fc = FullyConnected(name='akida_edge_layer',
                              units=num_labels * n,
                              activation=False)
    akida_model_test.add(layer_fc)
    
    akida_model_test.map(device)

    #akida_model_test.summary()
    
    akida_model_test.compile(num_weights=num_weights,
                     num_classes=num_labels,
                     learning_competition=0.1)

    batch_size = 8
    preds_ak = np.zeros(y_val.shape[0])
    num_batches_val = ceil(x_val.shape[0] / batch_size)
    num_batches = ceil(x_train.shape[0] / batch_size)
    
    
    start = timer()
    for i in range(num_batches):
        s = slice(i * batch_size, (i + 1) * batch_size)
        akida_model_test.fit(x_train[s], y_train[s].astype(np.int32))
    end = timer()
    
    
    floor_power = device.soc.power_meter.floor
    
    preds_val_ak = akida_model_test.predict(x_val, num_classes=num_labels)
    acc_val_ak = np.sum(preds_val_ak == y_val) / y_val.shape[0]
    print(f'Akida inference {len(x_train)} MFCC took {end-start:.2f} with neurons per class: s.\n', n)
    print(f"Akida validation set accuracy on initial data set: {100 * acc_val_ak:.2f} %", 'with neurons per class: ',n)
    print(f'The floor power with {n} neurons per class was: ', floor_power)
    print(f'Statistics with {n} neurons were: {floor_power:.2f} mW')
    print(akida_model_test.statistics)
    del akida_model_test


Akida inference 34972 MFCC took 380.86 with neurons per class: s.
 2
Akida validation set accuracy on initial data set: 93.23 % with neurons per class:  2
The floor power with 2 neurons per class was:  917.8812866210938
Statistics with 2 neurons were: 917.88 mW

Sequence HW/conv_0-separable_7
Average framerate = 369.67 fps
Last inference power range (mW):  Avg 1046.12 / Min 1045.00 / Max 1047.00 / Std 1.01 
Last inference energy consumed (mJ/frame): 2.83
Sequence SW/separable_8
Average framerate = 1108.27 fps
Last inference power range (mW):  Avg 920.33 / Min 917.00 / Max 940.00 / Std 4.01 
Last inference energy consumed (mJ/frame): 0.83
Sequence HW/akida_edge_layer
Average framerate = 3527.85 fps
Last inference power range (mW):  Avg 920.25 / Min 920.00 / Max 922.00 / Std 0.58 
Last inference energy consumed (mJ/frame): 0.26
Akida inference 34972 MFCC took 385.46 with neurons per class: s.
 4
Akida validation set accuracy on initial data set: 93.34 % with neurons per class:  4
The flo

Akida inference 34972 MFCC took 605.69 with neurons per class: s.
 90
Akida validation set accuracy on initial data set: 93.75 % with neurons per class:  90
The floor power with 90 neurons per class was:  916.09375
Statistics with 90 neurons were: 916.09 mW

Sequence HW/conv_0-separable_7
Average framerate = 369.67 fps
Sequence SW/separable_8
Average framerate = 1118.47 fps
Last inference power range (mW):  Avg 920.00 / Min 920.00 / Max 920.00 / Std 0.00 
Last inference energy consumed (mJ/frame): 0.82
Sequence HW/akida_edge_layer
Average framerate = 1313.40 fps
Last inference power range (mW):  Avg 930.83 / Min 929.00 / Max 931.00 / Std 0.56 
Last inference energy consumed (mJ/frame): 0.71
Akida inference 34972 MFCC took 632.48 with neurons per class: s.
 100
Akida validation set accuracy on initial data set: 93.82 % with neurons per class:  100
The floor power with 100 neurons per class was:  916.09375
Statistics with 100 neurons were: 916.09 mW

Sequence HW/conv_0-separable_7
Averag

RuntimeError: Error when programming layer 'akida_edge_layer': 758424 cannot fit in a 20 bits signed integer

In [9]:
# Hyperparameter search of "learning competition"

learning_comp = [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.8, 1]
num_labels = 12
num_weights = 300
device.soc.power_measurement_enabled = True

for idx, n in enumerate(learning_comp):
    
    akida_model_test = convert(quantized_model)

    akida_model_test.pop_layer()
    layer_fc = FullyConnected(name='akida_edge_layer',
                              units=num_labels * 200,
                              activation=False)
    akida_model_test.add(layer_fc)
    
    akida_model_test.map(device)

    #akida_model_test.summary()
    
    akida_model_test.compile(num_weights=num_weights,
                     num_classes=num_labels,
                     learning_competition=n)

    batch_size = 8
    preds_ak = np.zeros(y_val.shape[0])
    num_batches_val = ceil(x_val.shape[0] / batch_size)
    num_batches = ceil(x_train.shape[0] / batch_size)
    
    
    start = timer()
    for i in range(num_batches):
        s = slice(i * batch_size, (i + 1) * batch_size)
        akida_model_test.fit(x_train[s], y_train[s].astype(np.int32))
    end = timer()
    
    
    floor_power = device.soc.power_meter.floor
    
    preds_val_ak = akida_model_test.predict(x_val, num_classes=num_labels)
    acc_val_ak = np.sum(preds_val_ak == y_val) / y_val.shape[0]
    print(f'Akida inference {len(x_train)} MFCC took {end-start:.2f} with learning competition: s.\n', n)
    print(f"Akida validation set accuracy on initial data set: {100 * acc_val_ak:.2f} %", 'with learning competition: ',n)
    print(f'The floor power with {n} learning competition was: ', floor_power)
    print(f'Statistics with {n} learning competition were: {floor_power:.2f} mW')
    print(akida_model_test.statistics)
    del akida_model_test

Akida inference 34972 MFCC took 922.22 with learning competition: s.
 0.1
Akida validation set accuracy on initial data set: 93.89 % with learning competition:  0.1
The floor power with 0.1 learning competition was:  916.09375
Statistics with 0.1 learning competition were: 916.09 mW

Sequence HW/conv_0-separable_7
Average framerate = 369.67 fps
Sequence SW/separable_8
Average framerate = 1119.91 fps
Last inference power range (mW):  Avg 920.00 / Min 920.00 / Max 920.00 / Std 0.00 
Last inference energy consumed (mJ/frame): 0.82
Sequence HW/akida_edge_layer
Average framerate = 729.35 fps
Last inference power range (mW):  Avg 933.31 / Min 933.00 / Max 938.00 / Std 0.94 
Last inference energy consumed (mJ/frame): 1.28
Akida inference 34972 MFCC took 922.45 with learning competition: s.
 0.15
Akida validation set accuracy on initial data set: 93.94 % with learning competition:  0.15
The floor power with 0.15 learning competition was:  916.09375
Statistics with 0.15 learning competition wer

<font size="5"> 3. Edge Learning with previously defined Hyperparameters</font>

In [10]:
# Convert to an Akida model

akida_model = convert(quantized_model)
akida_model.summary()

# Replace the last layer by a classification layer with binary weights
# Here, we choose to set 15 neurons per class.

num_neurons_per_class = 200

akida_model.pop_layer()
layer_fc = FullyConnected(name='akida_edge_layer',
                          units=num_labels * num_neurons_per_class,
                          activation=False)
akida_model.add(layer_fc)

akida_model.map(device)

                 Model Summary                 
_______________________________________________
Input shape   Output shape  Sequences  Layers
[40, 101, 1]  [1, 1, 12]    1          10    
_______________________________________________

                SW/conv_0-dense (Software)                 
___________________________________________________________
Layer (type)             Output shape   Kernel shape     
conv_0 (InputConv.)      [51, 20, 32]   (3, 3, 1, 32)    
___________________________________________________________
separable_1 (Sep.Conv.)  [51, 20, 32]   (3, 3, 32, 1)    
___________________________________________________________
                                        (1, 1, 32, 32)   
___________________________________________________________
separable_2 (Sep.Conv.)  [26, 10, 64]   (3, 3, 32, 1)    
___________________________________________________________
                                        (1, 1, 32, 64)   
_______________________________________________________

In [11]:
akida_model.summary()

                 Model Summary                 
_______________________________________________
Input shape   Output shape  Sequences  Layers
[40, 101, 1]  [1, 1, 2400]  3          10    
_______________________________________________

     HW/conv_0-separable_7 (Hardware) - size: 204356 bytes     
_______________________________________________________________
Layer (type)             Output shape   Kernel shape      NPs
conv_0 (InputConv.)      [51, 20, 32]   (3, 3, 1, 32)     N/A
_______________________________________________________________
separable_1 (Sep.Conv.)  [51, 20, 32]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 32)       
_______________________________________________________________
separable_2 (Sep.Conv.)  [26, 10, 64]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 64)       
_______

In [12]:
# Print the performance of the Akida model

results = akida_model.predict(x_train)
accuracy = (y_train == results).mean()

print('SNN accuracy on training set after edge layer replacement:', accuracy * 100,'%')


results = akida_model.predict(x_val)
accuracy = (y_val == results).mean()

print('SNN accuracy on validation set after edge layer replacement:', accuracy * 100,'%')

results = akida_model.predict(x_test)
accuracy = (y_test == results).mean()


print('SNN accuracy on test set after edge layer replacement:', accuracy * 100,'%')

SNN accuracy on training set after edge layer replacement: 7.820542148004117 %
SNN accuracy on validation set after edge layer replacement: 7.778540379775794 %
SNN accuracy on test set after edge layer replacement: 8.053077099062 %


In [13]:
# Estimate the hyperparameter "number of weights".
# This hyperparameter describes the number of weights assigned for each neuron in the edge layer. 
# As described in the MetaTF documentation 10% of the training set is used to evaluate the sparsity. 
# Code Source: https://doc.brainchipinc.com/examples/edge/plot_1_edge_learning_kws.html

num_samples = ceil(0.1 * x_train.shape[0])
sparsities = evaluate_sparsity(akida_model, x_train[:num_samples])

# Retrieve the number of output spikes from the feature extractor output
output_density = 1 - sparsities[akida_model.get_layer('separable_8')]
avg_spikes = quantized_model.get_layer('separable_8').output_shape[-1] * output_density
print(f"Average number of spikes: {avg_spikes}")

# Fix the number of weights to 1.2 times the average number of output spikes
num_weights = int(1.2 * avg_spikes)
print("The number of weights is then set to:", num_weights)

Average number of spikes: 250.38078902229847
The number of weights is then set to: 300


In [14]:
# Compile Akida model with learning parameters

akida_model.compile(num_weights=num_weights,
                 num_classes=num_labels,
                 learning_competition=0.15)
akida_model.summary()

                 Model Summary                 
_______________________________________________
Input shape   Output shape  Sequences  Layers
[40, 101, 1]  [1, 1, 2400]  3          10    
_______________________________________________

     HW/conv_0-separable_7 (Hardware) - size: 204356 bytes     
_______________________________________________________________
Layer (type)             Output shape   Kernel shape      NPs
conv_0 (InputConv.)      [51, 20, 32]   (3, 3, 1, 32)     N/A
_______________________________________________________________
separable_1 (Sep.Conv.)  [51, 20, 32]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 32)       
_______________________________________________________________
separable_2 (Sep.Conv.)  [26, 10, 64]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 64)       
_______

In [15]:
# Train the Akida model
# Code Source: https://doc.brainchipinc.com/examples/edge/plot_1_edge_learning_kws.html

batch_size = 32
preds_ak = np.zeros(y_val.shape[0])
num_batches_val = ceil(x_val.shape[0] / batch_size)

# Train the last layer using Akida `fit` method
print(f"Akida learning with {num_labels} classes... \
        (this step can take a few minutes)")
num_batches = ceil(x_train.shape[0] / batch_size)
start = timer()
for i in range(num_batches):
    s = slice(i * batch_size, (i + 1) * batch_size)
    akida_model.fit(x_train[s], y_train[s].astype(np.int32))
end = timer()

print(f"Elapsed time for Akida training: {end-start:.2f} s")

Akida learning with 12 classes...         (this step can take a few minutes)
Elapsed time for Akida training: 365.64 s


In [16]:
# Print the performanc of the Akida model
# Code Source: https://doc.brainchipinc.com/examples/edge/plot_1_edge_learning_kws.html

preds_val_ak = akida_model.predict(x_train, num_classes=num_labels)
acc_val_ak = np.sum(preds_val_ak == y_train) / y_train.shape[0]
print(f"Akida train set accuracy: {100 * acc_val_ak:.2f} %")

# Measure Akida accuracy on validation set
preds_val_ak = np.zeros(y_val.shape[0])
for i in range(num_batches_val):
    s = slice(i * batch_size, (i + 1) * batch_size)
    preds_val_ak[s] = akida_model.predict(x_val[s], num_classes=num_labels)

acc_val_ak = np.sum(preds_val_ak == y_val) / y_val.shape[0]
print(f"Akida validation set accuracy: {100 * acc_val_ak:.2f} %")

# Measure Akida accuracy on validation set
preds_val_ak = np.zeros(y_test.shape[0])
for i in range(num_batches_val):
    s = slice(i * batch_size, (i + 1) * batch_size)
    preds_val_ak[s] = akida_model.predict(x_test[s], num_classes=num_labels)

acc_val_ak = np.sum(preds_val_ak == y_test) / y_test.shape[0]
print(f"Akida test set accuracy: {100 * acc_val_ak:.2f} %")

Akida train set accuracy: 99.02 %
Akida validation set accuracy: 94.07 %
Akida test set accuracy: 93.46 %


In [18]:
# Save the Akida model

akida_model.save('akida_model_edge.fbz')

In [19]:
# Add 3 classes

akida_model.add_classes(3)

In [20]:
# Train the Akida on the new keywords
# Code Source: https://doc.brainchipinc.com/examples/edge/plot_1_edge_learning_kws.html

print("\nEdge learning with 3 new classes ...")
start = timer()
akida_model.fit(x_train_edge, y_train_edge.astype(np.int32))
end = timer()
print(f"Elapsed time for Akida edge learning: {end-start:.2f} s")


Edge learning with 3 new classes ...
Elapsed time for Akida edge learning: 3.96 s


In [21]:
# Define the number of keywords used in the edge data set

num_labels_edge = 15

In [22]:
# Print the performance on the different data sets

preds_val_ak_old_train = akida_model.predict(x_train, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_old_train == y_train) / y_train.shape[0]
print(f"Akida train set accuracy on old data set: {100 * acc_val_ak:.2f} %")
preds_val_ak_old_val = akida_model.predict(x_val, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_old_val == y_val) / y_val.shape[0]
print(f"Akida validation set accuracy on old data set: {100 * acc_val_ak:.2f} %")
preds_val_ak_old_test = akida_model.predict(x_test, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_old_test == y_test) / y_test.shape[0]
print(f"Akida test set accuracy on old data set: {100 * acc_val_ak:.2f} %")

Akida train set accuracy on old data set: 98.97 %
Akida validation set accuracy on old data set: 94.01 %
Akida test set accuracy on old data set: 93.27 %


In [23]:
# Print the performance on the edge data set. 

preds_val_ak_new_train = akida_model.predict(x_train_edge, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_new_train == y_train_edge) / y_train_edge.shape[0]
print(f"Akida training set accuracy on new data set: {100 * acc_val_ak:.2f} %")
preds_val_ak_new_val = akida_model.predict(x_val_edge, num_classes=num_labels_edge)
acc_val_ak = np.sum(preds_val_ak_new_val == y_val_edge) / y_val_edge.shape[0]
print(f"Akida validation set accuracy on new data set: {100 * acc_val_ak:.2f} %")

Akida training set accuracy on new data set: 96.22 %
Akida validation set accuracy on new data set: 92.94 %
